# Projeto 1 - Ciência dos Dados

Nome: Enzo Davi

Nome: Antonio Anderson

Nome: Felipe Leventhal

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [47]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import unidecode
import nltk
from nltk import SnowballStemmer
from nltk.corpus import stopwords

In [48]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\enzod\Downloads\CDADOS\PROJETO 1


Carregando a base de dados com os tweets classificados manualmente:

In [49]:
train = pd.read_excel('dados_treino1.xlsx')
train.head(5)

,Mensagem,TARGET
0,"""Terror"" muito enrolado e cheio de detalhes de...",2
1,+,4
2,Caixa-Harry-Potter-Edição-Premium-Exclusiva-Am...,3
3,A caixa do box veio rasgada e amaçada; página ...,1
4,"A caixa veio estragada, queria deixar ela a mo...",1


In [50]:
test = pd.read_excel('dados_teste1.xlsx')
test.head(5)

,Mensagem,TARGET
0,"A história em si é boa, mas é bem cansativa. T...",2
1,O livro fala de um contexto bastante interessa...,2
2,"O livro é excelente no primeiro terço, porém, ...",2
3,"Decepção total com o livro, nem pareciam o cas...",2
4,"Muito difícil a leitura deste livro, acredito ...",2


___
## Classificador automático


Nosso modelo estuda uma base de dados que contém críticas a livros presentes na plataforma da Amazon. Sendo assim, nossa classificação tem como base 4 grupos: 
* Grupo 1: Críticas voltadas para a plataforma *Amazon* e os serviços que ela oferece.
* Grupo 2: Comentários que envolvem o *Autor* do livro.
* Grupo 3: Comentários que envolvem a *Editora* do livro.
* Grupo 4: *Outros*.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [51]:
train.head(3)

,Mensagem,TARGET
0,"""Terror"" muito enrolado e cheio de detalhes de...",2
1,+,4
2,Caixa-Harry-Potter-Edição-Premium-Exclusiva-Am...,3


In [52]:
train.TARGET.value_counts().sort_index()

1    115
2    271
3     79
4     35
Name: TARGET, dtype: int64

In [53]:
test.TARGET.value_counts().sort_index()

1     62
2    131
3     23
4     34
Name: TARGET, dtype: int64

In [54]:
def cleanup(text):
    
    #Remove as pontuações
    pontuacao = '[´"!-.:?;$'']' 
    pattern = re.compile(pontuacao)
    texto_limpo = re.sub(pattern, '', text)
    
    #Remove acentos
    texto_limpo = unidecode.unidecode(texto_limpo)
    #Deixa todas as letras minúsculas
    texto_limpo = texto_limpo.lower()
    
    return texto_limpo        

In [55]:
cleanup('bribaaa AAAA NÁAAAÁAA....')

'bribaaa aaaa naaaaaaa'

In [56]:
#Divide o texto em uma lista de palavras
def tokenize(texto):
    lista_texto = texto.split()
    return lista_texto

In [57]:
def stop_words(lista_texto):
    
    nltk.download('stopwords')
    stop_words = set(stopwords.words('portuguese'))
    new_list = list()
    for palavra in lista_texto:
        if palavra not in stop_words:
            new_list.append(palavra)

    return new_list

In [58]:
def stemming(lista_texto):
    stemmer = SnowballStemmer('portuguese')
    new_list = list()
    for palavra in lista_texto:
        new_list.append(stemmer.stem(palavra))
    
    return new_list

In [59]:
def vira_texto(s):
    texto = s.to_string(index=False)
    return texto

In [67]:
def roda_tudo(texto):
    texto1 = vira_texto(texto)
    texto2 = cleanup(texto1)
    texto3 = tokenize(texto2)
    texto4 = stop_words(texto3)
    texto5 = stemming(texto4)
    series_texto = pd.Series(texto5).to_frame()
    return series_texto

### Classificando os grupos:

In [68]:
Grupo1 = train.loc[train.TARGET == 1, :]

Grupo2 = train.loc[train.TARGET == 2, :]

Grupo3 = train.loc[train.TARGET == 3, :]

Grupo4 = train.loc[train.TARGET == 4, :]

In [74]:
vira_texto(Grupo1)

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                Mensagem  TARGET\n                                                                                                                                                                     

In [72]:
roda_tudo(Grupo1)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\enzod\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,0
0,mensag
1,target
2,caix
3,box
4,vei
...,...
2938,1
2939,vei
2940,amass
2941,pont


In [73]:
roda_tudo(Grupo2)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\enzod\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,0
0,mensag
1,target
2,terror
3,enrol
4,chei
...,...
9517,2
9518,vies
9519,ideolog
9520,gritant


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**